In [ ]:
!pip install transformers sentencepiece sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 9.3 MB/s 
     |████████████████████████████████| 1.3 MB 37.1 MB/s 
     |████████████████████████████████| 880 kB 55.7 MB/s 
     |████████████████████████████████| 120 kB 54.7 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1632737092488e979ec3a910902328b079bf4db2fea2514aa77dddeb8d27bd1d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
from transformers import *

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
# models we gonna use for this tutorial
model_names = [
  "tuner007/pegasus_paraphrase",
  "Vamsi/T5_Paraphrase_Paws",
  "prithivida/parrot_paraphraser_on_T5", # Parrot
]

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--tuner007--pegasus_paraphrase/snapshots/0159e2949ca73657a2f1329898f51b7bb53b9ab2/config.json
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "in

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--tuner007--pegasus_paraphrase/snapshots/0159e2949ca73657a2f1329898f51b7bb53b9ab2/pytorch_model.bin
All model checkpoint weights were used when initializing PegasusForConditionalGeneration.

All the weights of PegasusForConditionalGeneration were initialized from the model checkpoint at tuner007/pegasus_paraphrase.
If your task is similar to the task the model of the checkpoint was trained on, you can already use PegasusForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--tuner007--pegasus_paraphrase/snapshots/0159e2949ca73657a2f1329898f51b7bb53b9ab2/spiece.model
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--tuner007--pegasus_paraphrase/snapshots/0159e2949ca73657a2f1329898f51b7bb53b9ab2/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--tuner007--pegasus_paraphrase/snapshots/0159e2949ca73657a2f1329898f51b7bb53b9ab2/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--tuner007--pegasus_paraphrase/snapshots/0159e2949ca73657a2f1329898f51b7bb53b9ab2/config.json
Model config PegasusConfig {
  "_name_or_path": "tuner007/pegasus_paraphrase",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  

In [ ]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
sentence = "Learning is the process of acquiring new understanding, knowledge, behaviors, skills, values, attitudes, and preferences."

In [ ]:
get_paraphrased_sentences(model, tokenizer, sentence, num_beams=10, num_return_sequences=10)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 60 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['Learning involves the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes, and preferences.',
 'Learning is the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes, and preferences.',
 'The process of learning is the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes, and preferences.',
 'Gaining new understanding, knowledge, behaviors, skills, values, attitudes, and preferences is the process of learning.',
 'New understanding, knowledge, behaviors, skills, values, attitudes, and preferences are acquired through learning.',
 'Learning is the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes and preferences.',
 'The process of learning is the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes and preferences.',
 'New understanding, knowledge, behaviors, skills, values, attitudes, and preferences can be acquired through learning.',
 

In [ ]:
get_paraphrased_sentences(model, tokenizer, "To paraphrase a source, you have to rewrite a passage without changing the meaning of the original text.", num_beams=10, num_return_sequences=10)

['You have to keep the meaning of the original text in order to paraphrase a source.',
 'To paraphrase a source, you have to change the meaning of the original text.',
 'You need to keep the meaning of the original text in order to paraphrase a source.',
 'You have to change the meaning of the original text in order to paraphrase a source.',
 'You have to rewrite a passage without changing the meaning of the original text.',
 'You have to keep the original meaning of the text in order to paraphrase a source.',
 'Changing the meaning of the original text is not enough to paraphrase a source.',
 'You have to rewrite a passage without changing its meaning.',
 'You have to keep the original meaning of the text in order to paraphrase it.',
 'You have to keep the meaning of the original text in order to paraphrase it.']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Vamsi--T5_Paraphrase_Paws/snapshots/3bbf07dc42d5ddc9ca77c5589ce7239b0b731832/config.json
Model config T5Config {
  "_name_or_path": "Vamsi/T5_Paraphrase_Paws",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--Vamsi--T5_Paraphrase_Paws/snapshots/3bbf07dc42d5ddc9ca77c5589ce7239b0b731832/spiece.model
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--Vamsi--T5_Paraphrase_Paws/snapshots/3bbf07dc42d5ddc9ca77c5589ce7239b0b731832/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Vamsi--T5_Paraphrase_Paws/snapshots/3bbf07dc42d5ddc9ca77c5589ce7239b0b731832/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Vamsi--T5_Paraphrase_Paws/snapshots/3bbf07dc42d5ddc9ca77c5589ce7239b0b731832/config.json
Model config T5Config {
  "_name_or_path": "Vamsi/T5_Paraphrase_Paws",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "de

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--Vamsi--T5_Paraphrase_Paws/snapshots/3bbf07dc42d5ddc9ca77c5589ce7239b0b731832/pytorch_model.bin
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at Vamsi/T5_Paraphrase_Paws.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


In [ ]:
get_paraphrased_sentences(model, tokenizer, "paraphrase: " + "One of the best ways to learn is to teach what you've already learned")

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['One of the best ways to learn is to teach what you have already learned.',
 "One of the best ways to learn is to teach what you've already learned.",
 'One of the best ways to learn is to teach what you already know.',
 'One of the best ways to learn is to teach what you have already learned.',
 'One of the best ways to learn is to teach what you already learned.']

In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-2123wdr9
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-2123wdr9
     |████████████████████████████████| 50 kB 3.2 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8602 sha256=4f7d91f5d398c49661dc5418a4643a8c54881c627460b8fdcf79b01c164c4b66
  Stored in directory: /tmp/pip-ephem-wheel-cache-hs6zq7fv/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149866 sha256=87e3cbed48e5f79952e4c0f87e75f887d1bcd74f0da725a38dafbfd5a0cbdb45
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e

In [ ]:
from parrot import Parrot

parrot = Parrot()

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--prithivida--parrot_paraphraser_on_T5/snapshots/9f32aa1e456e8e8a90d97e8673365f3090fa49fa/config.json
Model config T5Config {
  "_name_or_path": "prithivida/parrot_paraphraser_on_T5",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
  

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_paraphrase-distilroberta-base-v2/config.json
Model config RobertaConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_paraphrase-distilroberta-base-v2/",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transforme

In [ ]:
phrases = [
  sentence,
  "One of the best ways to learn is to teach what you've already learned",
  "Paraphrasing is the process of coming up with someone else's ideas in your own words"
]

for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  paraphrases = parrot.augment(input_phrase=phrase)
  if paraphrases:
    for paraphrase in paraphrases:
      print(paraphrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  Learning is the process of acquiring new understanding, knowledge, behaviors, skills, values, attitudes, and preferences.
----------------------------------------------------------------------------------------------------
('learning is the process of acquiring new knowledge behaviors skills values attitudes and preferences', 27)
('learning is the process of learning understanding knowledge behavior skills values attitudes and preferences', 24)
----------------------------------------------------------------------------------------------------
Input_phrase:  One of the best ways to learn is to teach what you've already learned
----------------------------------------------------------------------------------------------------
('one of the best ways to learn is to teach what you know already', 25)
----------------------------------------------------------------------------